In [3]:
import mysql.connector
from dotenv import load_dotenv
import os

load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")

# MySQLデータベースに接続
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST", "localhost"),
    user=os.getenv("DB_USER", "stock"),
    password=os.getenv("DB_PASSWORD", "ryotaro1212"),
    database=os.getenv("DB_NAME", "stock")
)

cursor = conn.cursor()

# 更新したいカラムのリスト
columns = ["SP_500", "vix", "NASDAQ", 'NY_Dow', 'value']

# 各カラムについて処理を行う
for column in columns:
    # NULL値を含む行を取得
    cursor.execute(f"SELECT id, {column} FROM stock_dataset WHERE {column} IS NULL")
    rows = cursor.fetchall()

    for row in rows:
        current_id = row[0]
        
        # 前の行の値を取得
        cursor.execute(f"SELECT {column} FROM stock_dataset WHERE id = %s", (current_id - 1,))
        previous_row = cursor.fetchone()
        
        if previous_row:
            previous_value = previous_row[0]
            
            # NULL値を前の行の値で更新
            cursor.execute(f"UPDATE stock_dataset SET {column} = %s WHERE id = %s", (previous_value, current_id))
            conn.commit()

# 接続を閉じる
cursor.close()
conn.close()


In [2]:
import json
import mysql.connector
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# 環境変数を読み込む
load_dotenv()

# MySQLデータベースに接続
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST", "localhost"),
    user=os.getenv("DB_USER", "stock"),
    password=os.getenv("DB_PASSWORD", "ryotaro1212"),
    database=os.getenv("DB_NAME", "stock")
)

cursor = conn.cursor(dictionary=True)

with open('./scrape_data/stocks_values.json', 'r') as file:
    data = json.load(file)

def test_get_days(stock_code):
    query = """
    SELECT date, time
    FROM stock_dataset
    WHERE stock_code = %s
    AND value IS NULL
    """
    cursor.execute(query, (stock_code,))
    days = cursor.fetchall()
    dates = list(set(entry['date'].isoformat() for entry in days))
    return dates

def find_previous_value(stock_symbol, date_str, json_days, entries):
    date_obj = datetime.strptime(date_str, "%Y-%m-%d") - timedelta(days=1)
    previous_date_str = date_obj.strftime("%Y-%m-%d")

    for entry in entries:
        if entry['date'] == previous_date_str:
            return entry['close']

    return find_previous_value(stock_symbol, previous_date_str, json_days, entries)

for stock_symbol, entries in data.items():
    print(f'Stock Symbol: {stock_symbol}')

    days = test_get_days(stock_symbol)

    data_to_update = []

    json_days = list(set(entry['date'] for entry in entries))

    for date in days:
        if date not in json_days:
            update_value = find_previous_value(stock_symbol, date, json_days, entries)
            print(f'Missing date: {date}, filling with previous value: {update_value}')

            data = {
                'date': date,
                'column_name': 'value',
                'update_value': update_value.replace(',', ''),
                'stock_code': stock_symbol
            }
            data_to_update.append(data)
        else:
            for entry in entries:
                if entry['date'] == date:
                    update_value = entry['close']
                    print(f"Date: {date}, Close: {update_value}")

                    data = {
                        'date': date,
                        'column_name': 'value',
                        'update_value': update_value.replace(',', ''),
                        'stock_code': stock_symbol
                    }
                    data_to_update.append(data)
                    break

    for data in data_to_update:
        query = f"""
        UPDATE stock_dataset
        SET {data['column_name']} = %s
        WHERE date = %s AND stock_code = %s
        """

        try:
            cursor.execute(query, (data['update_value'], data['date'], data['stock_code']))
            conn.commit()
            print(f"Updated {cursor.rowcount} rows for date: {data['date']} and stock_code: {data['stock_code']}")
        except mysql.connector.Error as e:
            print(f"Error updating column: {e}")

cursor.close()
conn.close()


Stock Symbol: A
Stock Symbol: AAL
Stock Symbol: AAPL
Stock Symbol: ABBV
Stock Symbol: ABNB
Stock Symbol: ABT
Stock Symbol: ACGL
Stock Symbol: ACN


KeyboardInterrupt: 